__Purpose:__ Introduce Federated Learning, specifically by implementing FedAveraging on our dataset and moving on to more advanced methods.
<br>
1. Need to figure out what the weights are
2. Need to look into asynchronous FL
3. Create a global decoder from the last update... can I test it? I don't think so...
4. I still think it could be beneficial to create a random external model that we could easily do FL on...
5. Does scipy.optimize.minimize() run 1 iter or all necessary? Can it be replaced with SGD?  How are BFGS and SGD related?

In [2]:
import numpy as np
import pandas as pd
import time

In [ ]:
from experiment_params import *

## Code From Simulations

In [3]:
# set up gradient of cost:
# d(c_L2(D))/d(D) = 2*(DF + HV - V+)*F.T + 2*alphaD*D
def gradient_cost_l2(F, D, H, V, alphaF=1e-2, alphaD=1e-2):
    '''
    F: 64 channels x time EMG signals
    V: 2 x time target velocity
    D: 2 (x y vel) x 64 channels decoder # TODO: we now have a timeseries component - consult Sam
    H: 2 x 2 state transition matrix
    ''' 
    Nd = 2
    Ne = 64
    Nt = learning_batch

    # TODO: add depth (time) to D
    D = np.reshape(D,(Nd, Ne))
    Vplus = V[:,1:]
    Vminus = V[:,:-1]

    return ((2 * (D@F + H@Vminus - Vplus) @ F.T / (Nd*Nt) 
        + 2 * alphaD * D / (Nd*Ne)).flatten())

  # set up gradient of cost:
# d(c_L2(D))/d(D) = 2*(DF + HV - V+)*F.T + 2*alphaD*D
def gradient_cost_l2_discrete(F, D, H, V, alphaF=1e-2, alphaD=1e-2):
    '''
    F: 64 channels x time EMG signals
    V: 2 x time target velocity
    D: 2 (x y vel) x 64 channels decoder # TODO: we now have a timeseries component - consult Sam
    H: 2 x 2 state transition matrix
    ''' 
    Nd = 2
    Ne = 64
    Nt = learning_batch

    # TODO: add depth (time) to D
    D = np.reshape(D,(Nd, Ne))
    Vplus = V[:,1:]
    Vminus = V[:,:-1]
    v_unbounded = D@F
    theta = np.arctan2(v_unbounded[1],v_unbounded[0])
    v_actual = np.asarray([10*np.cos(theta),10*np.sin(theta)])
    return ((2 * (v_actual + H@Vminus - Vplus) @ F.T / (Nd*Nt) 
        + 2 * alphaD * D / (Nd*Ne)).flatten())
        
# set up the cost function: 
# c_L2 = (||DF + HV - V+||_2)^2 + alphaD*(||D||_2)^2 + alphaF*(||F||_2)^2
def cost_l2_discrete(F, D, H, V, alphaF=1e-2, alphaD=1e-2):
    '''
    F: 64 channels x time EMG signals
    V: 2 x time target velocity
    D: 2 (x y vel) x 64 channels decoder
    H: 2 x 2 state transition matrix
    ''' 
    Nd = 2
    Ne = 64 # default = 64
    Nt = learning_batch
    # TODO: add depth (time) to D
    D = np.reshape(D,(Nd,Ne))
    Vplus = V[:,1:]
    Vminus = V[:,:-1]
    v_unbounded = D@F
    theta = np.arctan2(v_unbounded[1],v_unbounded[0])
    v_actual = np.asarray([10*np.cos(theta),10*np.sin(theta)])
    e = ( np.sum( (v_actual + H@Vminus - Vplus)**2 ) / (Nd*Nt) 
            + alphaD * np.sum( D**2 ) / (Nd*Ne)
            + alphaF * np.sum( F**2 ) / (Ne*Nt) )
    return e

# set up the cost function: 
# c_L2 = (||DF + HV - V+||_2)^2 + alphaD*(||D||_2)^2 + alphaF*(||F||_2)^2
def cost_l2(F, D, H, V, alphaF=1e-2, alphaD=1e-2):
    '''
    F: 64 channels x time EMG signals
    V: 2 x time target velocity
    D: 2 (x y vel) x 64 channels decoder
    H: 2 x 2 state transition matrix
    ''' 
    Nd = 2
    Ne = 64 # default = 64
    Nt = learning_batch
    # TODO: add depth (time) to D
    D = np.reshape(D,(Nd,Ne))
    Vplus = V[:,1:]
    Vminus = V[:,:-1]

    e = ( np.sum( (D @ F + H@Vminus - Vplus)**2 ) / (Nd*Nt) 
            + alphaD * np.sum( D**2 ) / (Nd*Ne)
            + alphaF * np.sum( F**2 ) / (Ne*Nt) )
    return e

def estimate_decoder(F, H, V):
    return (V[:,1:]-H@V[:,:-1])@np.linalg.pinv(F)

In [4]:
def simulation(D,learning_batch,alpha,alphaF=1e-2,alphaD=1e-2):
    p_classify = []
    accuracy_temp = []
    num_updates = int(np.floor((filtered_signals.shape[0]-1)/learning_batch)) # how many times can we update decoder based on learning batch    

    # RANDOMIZE DATASET
    randomized_integers = np.random.permutation(range(0,cued_target_position.shape[0]))
    filtered_signals_randomized = filtered_signals[randomized_integers]
    cued_target_position_randomized = cued_target_position[randomized_integers]
    # batches the trials into each of the update batch
    for ix in range(num_updates):
        s = np.hstack([x for x in filtered_signals_randomized[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:,:]])# stack s (64 x (60 timepoints x learning batch size))
        p_intended = np.hstack([np.tile(x[:,np.newaxis],60) for x in cued_target_position_randomized[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:]]) # stack p_intended (2 x 60 timepoints x learning batch size)
        v_intended,p_constrained = output_new_decoder(s,D[-1],p_intended)

        # CLASSIFY CURRENT DECODER ACCURACY
        v_actual = D[-1]@s
        for trial in range(learning_batch):
            v_trial = v_actual[:,int(trial*60):int((trial+1)*60)] # velocities for each trials (2,60)
            p_final = np.sum(v_trial,axis=1)[:,np.newaxis] # final position after integration (2,)
            p_classify.append(classify(p_final))
        
        # UPDATE DECODER
        u = copy.deepcopy(s) # u is the person's signal s (64 CHANNELS X TIMEPOINTS)
        q = copy.deepcopy(v_intended) # use cued positions as velocity vectors for updating decoder should be 2 x num_trials

        # emg_windows against intended_targets (trial specific cued target)
        F = copy.deepcopy(u[:,:-1]) # note: truncate F for estimate_decoder
        V = copy.deepcopy(q)

        # initial decoder estimate for gradient descent
        D0 = np.random.rand(2,64)

        # set alphas
        H = np.zeros((2,2))
        # use scipy minimize for gradient descent and provide pre-computed analytical gradient for speed
        out = minimize(lambda D: cost_l2(F,D,H,V), D0, method='BFGS', jac = lambda D: gradient_cost_l2(F,D,H,V), options={'disp': False})

        # reshape to decoder parameters
        W_hat = np.reshape(out.x,(2, 64))

        # DO SMOOTHBATCH
        W_new = alpha*D[-1] + ((1 - alpha) * W_hat)
        D.append(W_new)

        # COMPUTE CLASSIFICATION ACURACY 
        p_target = (cued_target_position[randomized_integers])[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:] # obtain target
        accuracy_temp.append(classification_accuracy(p_target,p_classify[-learning_batch:]))

    p_classify = np.asarray(p_classify)
    return accuracy_temp,D,p_constrained

# Modifying Simulations Code

In [5]:
# Added 2 new parameters
def simulation(D,learning_batch,alpha,alphaF=1e-2,alphaD=1e-2,display_info=False,num_iters=False):
    p_classify = []
    accuracy_temp = []
    num_updates = int(np.floor((filtered_signals.shape[0]-1)/learning_batch)) # how many times can we update decoder based on learning batch    

    # RANDOMIZE DATASET
    randomized_integers = np.random.permutation(range(0,cued_target_position.shape[0]))
    filtered_signals_randomized = filtered_signals[randomized_integers]
    cued_target_position_randomized = cued_target_position[randomized_integers]
    # batches the trials into each of the update batch
    for ix in range(num_updates):
        s = np.hstack([x for x in filtered_signals_randomized[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:,:]])# stack s (64 x (60 timepoints x learning batch size))
        p_intended = np.hstack([np.tile(x[:,np.newaxis],60) for x in cued_target_position_randomized[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:]]) # stack p_intended (2 x 60 timepoints x learning batch size)
        v_intended,p_constrained = output_new_decoder(s,D[-1],p_intended)

        # CLASSIFY CURRENT DECODER ACCURACY
        v_actual = D[-1]@s
        for trial in range(learning_batch):
            v_trial = v_actual[:,int(trial*60):int((trial+1)*60)] # velocities for each trials (2,60)
            p_final = np.sum(v_trial,axis=1)[:,np.newaxis] # final position after integration (2,)
            p_classify.append(classify(p_final))
        
        # UPDATE DECODER
        u = copy.deepcopy(s) # u is the person's signal s (64 CHANNELS X TIMEPOINTS)
        q = copy.deepcopy(v_intended) # use cued positions as velocity vectors for updating decoder should be 2 x num_trials

        # emg_windows against intended_targets (trial specific cued target)
        F = copy.deepcopy(u[:,:-1]) # note: truncate F for estimate_decoder
        V = copy.deepcopy(q)

        # initial decoder estimate for gradient descent
        D0 = np.random.rand(2,64)

        # set alphas
        H = np.zeros((2,2))
        # use scipy minimize for gradient descent and provide pre-computed analytical gradient for speed
        if num_iters is False:
            out = minimize(lambda D: cost_l2(F,D,H,V), D0, method='BFGS', jac = lambda D: gradient_cost_l2(F,D,H,V), options={'disp': display_info})
        else:
            out = minimize(lambda D: cost_l2(F,D,H,V), D0, method='BFGS', jac = lambda D: gradient_cost_l2(F,D,H,V), options={'disp': display_info, 'maxiter':num_iters})
        
        # reshape to decoder parameters
        W_hat = np.reshape(out.x,(2, 64))

        # DO SMOOTHBATCH
        W_new = alpha*D[-1] + ((1 - alpha) * W_hat)
        D.append(W_new)

        # COMPUTE CLASSIFICATION ACURACY 
        p_target = (cued_target_position[randomized_integers])[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:] # obtain target
        accuracy_temp.append(classification_accuracy(p_target,p_classify[-learning_batch:]))

    p_classify = np.asarray(p_classify)
    return accuracy_temp,D,p_constrained

In [11]:
# This requires the aopy package...
'''
weiner_task_data, weiner_config = aopy.data.load_bmi3d_hdf_table('',filename, 'weiner')
f = weiner_task_data
print(f.dtype.names) # get the names of the fields

timestamp = f['timestamp']
decoded_cursor_position = f['decoded_cursor_position']
decoded_velocity = f['decoded_velocity']
weiner_filter_w = f['weiner_filter_w']
weiner_filter_h = f['weiner_filter_h']
alpha = f['alpha']
raw_emg = f['raw_emg']
filtered_emg = f['filtered_emg']
reference = f['reference']
cued_target_position = f['cued_target_position']

# change target positions to angle
cued_angles = np.arctan2(cued_target_position[:,1], cued_target_position[:,0])
cued_angles_classes = (cued_angles*2/np.pi+2.001).astype(int) # need to add that 0.001 so all the numbers can round down during int conversion
'''

# Suppress output
;

''

In [ ]:
# I think the data I have is already the filtered EMG data?

#win_downsample = int(np.floor(raw_emg.shape[2]/2048*60))
#filtered_signals = np.zeros((50, 64,  win_downsample))
#window = np.floor(2048/60)
#for trial in range(1,raw_emg.shape[0]):
#    for ix in range(win_downsample - 1):
#        # window the signal
#        window_signal = raw_emg[trial,:,int(ix*window):int((ix+1)*window)]
#        # print(window_signal[0,0])
#        # filter the windowed signal
#        filtered_signal = filter_signal(window_signal)
#        filtered_signals[trial,:,ix] = np.squeeze(filtered_signal)
#        # print(filtered_signal)

#Therefore, import my pandas data
t0 = time.time()

emg_data_df1 = pd.read_csv("Data\emg_full_data1.csv")
emg_labels_df1 = pd.read_csv("Data\emg_full_labels1.csv")
emg_data_df2 = pd.read_csv("Data\emg_full_data2.csv")
emg_labels_df2 = pd.read_csv("Data\emg_full_labels2.csv")

t1 = time.time()
total = t1-t0  
print(total)

display(emg_data_df1.head())
display(emg_labels_df1.head())

In [ ]:
num_targets = 4
TARGET_LOCATION_RADIUS = 10
thetas = 2*np.pi/num_targets*np.arange(0,num_targets) # convert number of targets to angles
target_positions = TARGET_LOCATION_RADIUS*np.asarray([np.cos(thetas),np.sin(thetas)]).T

In [6]:
learning_batch = 8
alpha = .95 # higher alpha means more old decoder (slower update)
alphaF=1e-1
alphaD = 1e-1
# create random decoder 
D_0 = np.random.rand(2,64)
D = []
D_constant = []
D_bounded = []
D_constant_bounded = []
D.append(D_0)
D_constant.append(D_0)
D_bounded.append(D_0)
D_constant_bounded.append(D_0)
accuracy = []
accuracy_constant = []
accuracy_bounded = []
accuracy_constant_bounded = []

In [7]:
# Original code for running simulations...
# for ix in range(10000):
    #accuracy_constant_,D_constant,p_constrained_constant = simulation_constant_intent(D_constant,learning_batch,alpha,alphaF=alphaF,alphaD=alphaD)
    #accuracy_constant.extend(accuracy_constant_)
    #accuracy_,D,p_constrained = simulation(D,learning_batch,alpha,alphaF=alphaF,alphaD=alphaD)    
    #accuracy.extend(accuracy_)
    #accuracy_bounded_,D_bounded,p_bounded = simulation_bounded_pos(D_bounded,learning_batch,alpha,alphaF=alphaF,alphaD=alphaD)  
    #accuracy_bounded.extend(accuracy_bounded_)
    #accuracy_constant_bounded_,D_constant_bounded,p_constant_bounded = simulation_constant_intent_bounded(D_constant_bounded,learning_batch,alpha,alphaF=alphaF,alphaD=alphaD)
    #accuracy_constant_bounded.extend(accuracy_constant_bounded_)

In [8]:
# Modified code for running simulations...
# Why loop at all right now...
#for ix in range(10):
    #accuracy_,D,p_constrained = simulation(D,learning_batch,alpha,alphaF=alphaF,alphaD=alphaD)    


In [9]:
# Added 2 new parameters
#def simulation(D,learning_batch,alpha,alphaF=1e-2,alphaD=1e-2,display_info=False,num_iters=False):
#D  # Already defined
#learning_batch  # Already defined
#alpha  # Already defined
#alphaF=1e-2  #defined as something else earlier...
#alphaD=1e-2  #defined as something else earlier...
display_info=True
num_iters=False

######################################################################################
# Where do filter_signals and cued_target_position come from lol


p_classify = []
accuracy_temp = []
num_updates = int(np.floor((filtered_signals.shape[0]-1)/learning_batch)) # how many times can we update decoder based on learning batch    

# RANDOMIZE DATASET
randomized_integers = np.random.permutation(range(0,cued_target_position.shape[0]))
filtered_signals_randomized = filtered_signals[randomized_integers]
cued_target_position_randomized = cued_target_position[randomized_integers]
# batches the trials into each of the update batch
for ix in range(num_updates):
    s = np.hstack([x for x in filtered_signals_randomized[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:,:]])# stack s (64 x (60 timepoints x learning batch size))
    p_intended = np.hstack([np.tile(x[:,np.newaxis],60) for x in cued_target_position_randomized[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:]]) # stack p_intended (2 x 60 timepoints x learning batch size)
    v_intended,p_constrained = output_new_decoder(s,D[-1],p_intended)

    # CLASSIFY CURRENT DECODER ACCURACY
    v_actual = D[-1]@s
    for trial in range(learning_batch):
        v_trial = v_actual[:,int(trial*60):int((trial+1)*60)] # velocities for each trials (2,60)
        p_final = np.sum(v_trial,axis=1)[:,np.newaxis] # final position after integration (2,)
        p_classify.append(classify(p_final))

    # UPDATE DECODER
    u = copy.deepcopy(s) # u is the person's signal s (64 CHANNELS X TIMEPOINTS)
    q = copy.deepcopy(v_intended) # use cued positions as velocity vectors for updating decoder should be 2 x num_trials

    # emg_windows against intended_targets (trial specific cued target)
    F = copy.deepcopy(u[:,:-1]) # note: truncate F for estimate_decoder
    V = copy.deepcopy(q)

    # initial decoder estimate for gradient descent
    D0 = np.random.rand(2,64)

    # set alphas
    H = np.zeros((2,2))
    # use scipy minimize for gradient descent and provide pre-computed analytical gradient for speed
    if num_iters is False:
        out = minimize(lambda D: cost_l2(F,D,H,V), D0, method='BFGS', jac = lambda D: gradient_cost_l2(F,D,H,V), options={'disp': display_info})
    else:
        out = minimize(lambda D: cost_l2(F,D,H,V), D0, method='BFGS', jac = lambda D: gradient_cost_l2(F,D,H,V), options={'disp': display_info, 'maxiter':num_iters})

    # reshape to decoder parameters
    W_hat = np.reshape(out.x,(2, 64))

    # DO SMOOTHBATCH
    W_new = alpha*D[-1] + ((1 - alpha) * W_hat)
    D.append(W_new)

    # COMPUTE CLASSIFICATION ACURACY 
    p_target = (cued_target_position[randomized_integers])[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:] # obtain target
    accuracy_temp.append(classification_accuracy(p_target,p_classify[-learning_batch:]))

p_classify = np.asarray(p_classify)
#return accuracy_temp,D,p_constrained

NameError: name 'filtered_signals' is not defined